In [25]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fraudfinder - Inference Demo (New Feature Store)

## Overview

[Fraudfinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the Fraudfinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

This notebook shows how to use Feature Store and Model Endpoint for Realtime Inference

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)

Steps performed in this notebook:

* Invoke a Vetex AI Feature Store and Vetex AI Endpoint to test Realtime predictions

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [26]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)


BUCKET_NAME          = "qwiklabs-asl-02-111b5e486eb8-fraudfinder"
PROJECT              = "qwiklabs-asl-02-111b5e486eb8"
REGION               = "us-central1"
ID                   = "ytdm4"
FEATURESTORE_ID      = "fraudfinder_ytdm4"
MODEL_NAME           = "ff_model"
ENDPOINT_NAME        = "ff_model_endpoint"
TRAINING_DS_SIZE     = "1000"



### Import libraries and define constants

#### Libraries
Next you will import the libraries needed for this notebook. 

#### Variables

#### Initialize the Vertex AI SDK
Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [27]:
import json

# General imports
import os
import random
import sys
from datetime import datetime, timedelta

# from google.cloud import aiplatform
from google.cloud import aiplatform as vertex_ai
from google.cloud import bigquery
from google.cloud.aiplatform_v1 import (
    FeatureOnlineStoreAdminServiceClient,
    FeatureOnlineStoreServiceClient,
    FeatureRegistryServiceClient,
)
from google.cloud.aiplatform_v1.types import feature as feature_pb2
from google.cloud.aiplatform_v1.types import feature_group as feature_group_pb2
from google.cloud.aiplatform_v1.types import (
    feature_online_store as feature_online_store_pb2,
)
from google.cloud.aiplatform_v1.types import (
    feature_online_store_admin_service as feature_online_store_admin_service_pb2,
)
from google.cloud.aiplatform_v1.types import (
    feature_online_store_service as feature_online_store_service_pb2,
)
from google.cloud.aiplatform_v1.types import (
    feature_registry_service as feature_registry_service_pb2,
)
from google.cloud.aiplatform_v1.types import feature_view as feature_view_pb2
from google.cloud.aiplatform_v1.types import (
    featurestore_service as featurestore_service_pb2,
)
from google.cloud.aiplatform_v1.types import io as io_pb2

In [28]:
# Vertex AI SDK
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

### Define helper methods

In [29]:
def read_from_sub(project_id, subscription_name, messages=10):
    """
    Read messages from a Pub/Sub subscription
    Args:
        project_id: project ID
        subscription_name: the name of a Pub/Sub subscription in your project
        messages: number of messages to read
    Returns:
        msg_data: list of messages in your Pub/Sub subscription as a Python dictionary
    """

    import ast

    from google.api_core import retry
    from google.cloud import pubsub_v1

    subscriber = pubsub_v1.SubscriberClient()
    subscription_path = subscriber.subscription_path(
        project_id, subscription_name
    )

    # Wrap the subscriber in a 'with' block to automatically call close() to
    # close the underlying gRPC channel when done.
    with subscriber:
        # The subscriber pulls a specific number of messages. The actual
        # number of messages pulled may be smaller than max_messages.
        response = subscriber.pull(
            subscription=subscription_path,
            max_messages=messages,
            retry=retry.Retry(deadline=300),
        )

        if len(response.received_messages) == 0:
            print("no messages")
            return

        ack_ids = []
        msg_data = []
        for received_message in response.received_messages:
            msg = ast.literal_eval(
                received_message.message.data.decode("utf-8")
            )
            msg_data.append(msg)
            ack_ids.append(received_message.ack_id)

        # Acknowledges the received messages so they will not be sent again.
        subscriber.acknowledge(subscription=subscription_path, ack_ids=ack_ids)

        print(
            f"Received and acknowledged {len(response.received_messages)} messages from {subscription_path}."
        )

        return msg_data

#### Retrieve your Vertex AI endpoint name

In [30]:
endpoints = vertex_ai.Endpoint.list(
    filter=f"display_name={ENDPOINT_NAME}",  # optional: filter by specific endpoint name
    order_by="update_time",
)

ENDPOINT_ID = endpoints[-1].name
print(ENDPOINT_ID)

7142281298986401792


In [31]:
print(ENDPOINT_ID)
from google.cloud import aiplatform as aiplatform

# Instantiate Vertex AI Endpoint object
endpoint_obj = aiplatform.Endpoint(ENDPOINT_ID)

7142281298986401792


In [32]:
import time
messages_tx="no messages"
while messages_tx=="no messages":
    messages_tx = read_from_sub(
        project_id=PROJECT_ID, subscription_name="ff-tx-sub", messages=1
    )
    print(messages_tx)
    time.sleep(5) # Sleep for 5 seconds

messages_tx

Received and acknowledged 1 messages from projects/qwiklabs-asl-02-111b5e486eb8/subscriptions/ff-tx-sub.
[{'TX_ID': 'd911d4f9b4627ed5bd006ffff50fa2cf6bb4400c', 'TX_TS': '2025-10-08 20:20:07', 'CUSTOMER_ID': '2226055229115167', 'TERMINAL_ID': '10096253', 'TX_AMOUNT': 75.5}]


[{'TX_ID': 'd911d4f9b4627ed5bd006ffff50fa2cf6bb4400c',
  'TX_TS': '2025-10-08 20:20:07',
  'CUSTOMER_ID': '2226055229115167',
  'TERMINAL_ID': '10096253',
  'TX_AMOUNT': 75.5}]

In [33]:
API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

# Instantiate Vertex AI Feature Store object

data_client = FeatureOnlineStoreServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

In [34]:
import pprint

pp = pprint.PrettyPrinter(compact=True)


def fs_features_lookup(ff_feature_store, features_type, features_key):

    FEATURE_VIEW_ID = f"fv_fraudfinder_{features_type}"
    FEATURE_VIEW_FULL_ID = f"projects/{PROJECT_ID}/locations/{REGION}/featureOnlineStores/{ff_feature_store}/featureViews/{FEATURE_VIEW_ID}"

    features_map = {}

    print(FEATURE_VIEW_FULL_ID)

    try:
        fe_continuous_data = data_client.fetch_feature_values(
            request=feature_online_store_service_pb2.FetchFeatureValuesRequest(
                feature_view=FEATURE_VIEW_FULL_ID,
                data_key=feature_online_store_service_pb2.FeatureViewDataKey(
                    key=features_key
                ),
                data_format=feature_online_store_service_pb2.FeatureViewDataFormat.PROTO_STRUCT,
            )
        )
        features_map.update(
            {k: v for k, v in fe_continuous_data.proto_struct.items()}
        )
    except Exception as exp:
        print(f"Requested entity {features_key} was not found")
    return features_map

In [35]:
%%bigquery df --project {PROJECT_ID}
SELECT * FROM tx.t_customers_streaming_features
ORDER BY feature_timestamp DESC
LIMIT 1

Query is running:   0%|          |

Downloading:   0%|          |

In [36]:
df

,entity_id,feature_timestamp,customer_id_nb_tx_15min_window,customer_id_nb_tx_30min_window,customer_id_nb_tx_60min_window,customer_id_avg_amount_15min_window,customer_id_avg_amount_30min_window,customer_id_avg_amount_60min_window
0,0743615767114752,2025-10-08 20:22:00+00:00,1,1,1,60.39,60.39,60.39


In [37]:
customer_key = df["entity_id"][0]  # Or put known customer id here
print(f"entity_id={customer_key}")

entity_id=0743615767114752


In [38]:
customer_features = fs_features_lookup(
    FEATURESTORE_ID, "customers", customer_key  # customer_key
)
pp.pprint(customer_features)

projects/qwiklabs-asl-02-111b5e486eb8/locations/us-central1/featureOnlineStores/fraudfinder_ytdm4/featureViews/fv_fraudfinder_customers
{'customer_id_avg_amount_14day_window': 55.605625,
 'customer_id_avg_amount_15min_window': 49.79,
 'customer_id_avg_amount_1day_window': 55.115,
 'customer_id_avg_amount_30min_window': 49.79,
 'customer_id_avg_amount_60min_window': 49.79,
 'customer_id_avg_amount_7day_window': 56.754,
 'customer_id_nb_tx_14day_window': 16.0,
 'customer_id_nb_tx_15min_window': 1.0,
 'customer_id_nb_tx_1day_window': 2.0,
 'customer_id_nb_tx_30min_window': 1.0,
 'customer_id_nb_tx_60min_window': 1.0,
 'customer_id_nb_tx_7day_window': 15.0}


In [39]:
%%bigquery df --project {PROJECT_ID}
SELECT * FROM tx.v_terminals_features
ORDER BY feature_timestamp DESC
LIMIT 1

Query is running:   0%|          |

Downloading:   0%|          |

In [40]:
terminal_key = df["entity_id"][0]  # Or put known customer id here
print(f"entity_id={customer_key}")
terminal_features = fs_features_lookup(
    FEATURESTORE_ID, "terminals", terminal_key
)  # Change key values
pp.pprint(terminal_features)

entity_id=0743615767114752
projects/qwiklabs-asl-02-111b5e486eb8/locations/us-central1/featureOnlineStores/fraudfinder_ytdm4/featureViews/fv_fraudfinder_terminals
{'terminal_id_avg_amount_15min_window': 9.94,
 'terminal_id_avg_amount_30min_window': 9.94,
 'terminal_id_avg_amount_60min_window': 31.064999999999998,
 'terminal_id_nb_tx_14day_window': 142.0,
 'terminal_id_nb_tx_15min_window': 1.0,
 'terminal_id_nb_tx_1day_window': 13.0,
 'terminal_id_nb_tx_30min_window': 1.0,
 'terminal_id_nb_tx_60min_window': 2.0,
 'terminal_id_nb_tx_7day_window': 142.0,
 'terminal_id_risk_14day_window': 0.0,
 'terminal_id_risk_1day_window': 0.0,
 'terminal_id_risk_7day_window': 0.0}


In [41]:
import pprint

pp = pprint.PrettyPrinter(compact=True)

# Payload for manual test:
# payload_json = {
#     "TX_ID": "61210be0744c43232990152d3eb2c2deb6035d8b",
#     "TX_TS": "2025-09-06 17:27:51",
#     "CUSTOMER_ID": "7389471951168361",
#     "TERMINAL_ID": "45087784",
#     "TX_AMOUNT": 32.77
# }

default_features = {
    "customer_id_avg_amount_14day_window": 0,
    "customer_id_avg_amount_15min_window": 0,
    "customer_id_avg_amount_1day_window": 0,
    "customer_id_avg_amount_30min_window": 0,
    "customer_id_avg_amount_60min_window": 0,
    "customer_id_avg_amount_7day_window": 0,
    "customer_id_nb_tx_14day_window": 0,
    "customer_id_nb_tx_7day_window": 0,
    "customer_id_nb_tx_15min_window": 0,
    "customer_id_nb_tx_1day_window": 0,
    "customer_id_nb_tx_30min_window": 0,
    "customer_id_nb_tx_60min_window": 0,
    "terminal_id_avg_amount_15min_window": 0,
    "terminal_id_avg_amount_30min_window": 0,
    "terminal_id_avg_amount_60min_window": 0,
    "terminal_id_nb_tx_14day_window": 0,
    "terminal_id_nb_tx_15min_window": 0,
    "terminal_id_nb_tx_1day_window": 0,
    "terminal_id_nb_tx_30min_window": 0,
    "terminal_id_nb_tx_60min_window": 0,
    "terminal_id_nb_tx_7day_window": 0,
    "terminal_id_risk_14day_window": 0,
    "terminal_id_risk_1day_window": 0,
    "terminal_id_risk_7day_window": 0,
}

# Reading 1-st message
payload_json = messages_tx[0]

payload = default_features
payload["tx_amount"] = payload_json["TX_AMOUNT"]

# look up the customer features from New Vertex AI Feature Store
customer_features = fs_features_lookup(
    FEATURESTORE_ID, "customers", payload_json["CUSTOMER_ID"]
)
# print the customer features from Vertex AI Feature Store
print("-------------------------------------------------------")
print("customer_features:")
pp.pprint(customer_features)

# look up the treminal features from New Vertex AI Feature Store
terminal_features = fs_features_lookup(
    FEATURESTORE_ID, "terminals", payload_json["TERMINAL_ID"]
)
print("-------------------------------------------------------")
print("terminal features:")
pp.pprint(terminal_features)

# Add customer features to payload
payload.update(customer_features)

# Add terminal features to payload
payload.update(terminal_features)

# del payload["feature_timestamp"]

print("-------------------------------------------------------")
print("[Payload to be sent to Vertex AI endpoint]")
pp.pprint(payload)
print("-------------------------------------------------------")

result = endpoint_obj.predict(instances=[payload])

print("-------------------------------------------------------")
pp.pprint(f"[Prediction result]: {result}")
print("-------------------------------------------------------")

projects/qwiklabs-asl-02-111b5e486eb8/locations/us-central1/featureOnlineStores/fraudfinder_ytdm4/featureViews/fv_fraudfinder_customers
-------------------------------------------------------
customer_features:
{'customer_id_avg_amount_14day_window': 73.89888888888889,
 'customer_id_avg_amount_15min_window': 75.5,
 'customer_id_avg_amount_1day_window': 77.842,
 'customer_id_avg_amount_30min_window': 75.5,
 'customer_id_avg_amount_60min_window': 75.5,
 'customer_id_avg_amount_7day_window': 75.8,
 'customer_id_nb_tx_14day_window': 27.0,
 'customer_id_nb_tx_15min_window': 1.0,
 'customer_id_nb_tx_1day_window': 5.0,
 'customer_id_nb_tx_30min_window': 1.0,
 'customer_id_nb_tx_60min_window': 1.0,
 'customer_id_nb_tx_7day_window': 21.0}
projects/qwiklabs-asl-02-111b5e486eb8/locations/us-central1/featureOnlineStores/fraudfinder_ytdm4/featureViews/fv_fraudfinder_terminals
-------------------------------------------------------
terminal features:
{'terminal_id_avg_amount_15min_window': 75.5,
 't